In [175]:
import requests
import json
import pandas as pd

In [174]:
# Baixa o arquivo
json_uf = requests.get('https://xx9p7hp1p7.execute-api.us-east-1.amazonaws.com/prod/PortalMapa',
                       headers={'x-parse-application-id': 'unAFkcaNDeXajurGB7LChj8SgQYS2ptm'}).content

dados_uf = json.loads(json_uf)['results']


In [176]:
# Importa dados e colunas
df = pd.DataFrame(dados_uf)

# Ajusta horário
df['updatedAt'] = pd.to_datetime(df.updatedAt)
df['date'] = df['updatedAt'].dt.date.astype(str)
df['hour'] = df['updatedAt'].dt.hour.astype(str)
df['minute'] = df['updatedAt'].dt.minute.astype(str)
df['time'] =  df['hour'].str.cat(df['minute'], sep=':')

# Remove colunas desnecessárias
df = df.drop(['objectId', 'latitude', 'longitude','updatedAt',
              'createdAt', 'percent','letalidade','hour','minute'], axis=1)

df

,nome,qtd_confirmado,qtd_obito,date,time
0,Acre,42,0,2020-03-30,19:42
1,Alagoas,17,0,2020-03-30,19:43
2,Amapá,8,0,2020-03-30,19:44
3,Amazonas,151,1,2020-03-30,19:43
4,Bahia,176,1,2020-03-30,19:44
5,Ceará,372,5,2020-03-30,19:46
6,Distrito Federal,312,1,2020-03-30,19:45
7,Espírito Santo,72,0,2020-03-30,19:45
8,Goiás,61,1,2020-03-30,19:46
9,Maranhão,23,1,2020-03-30,19:45


In [168]:
# Troca nome dos estados para sigla
dicionario = {"Acre": "AC", "Alagoas": "AL", "Amapá": "AP", "Amazonas": "AM", "Bahia": "BA", "Ceará": "CE",
             "Distrito Federal": "DF", "Espírito Santo": "ES", "Goiás": "GO", "Maranhão": "MA",
              "Mato Grosso": "MT","Mato Grosso do Sul": "MS","Minas Gerais":"MG", "Paraná": "PR",
              "Paraíba": "PB", "Pará": "PA", "Pernambuco": "PE", "Piauí": "PI", "Rio Grande do Norte": "RN", 
              "Rio Grande do Sul": "RS", "Rio de Janeiro": "RJ", "Rondônia": "RO", "Roraima": "RR",
              "São Paulo": "SP", "Santa Catarina": "SC", "Sergipe": "SE", "Tocantins": "TO"
             }

df = df.replace({"nome": dicionario})


In [169]:
df.rename(columns={'nome':'uf', 'qtd_obito':'deaths', 'qtd_confirmado':'cases'}, inplace=True)

df

,uf,cases,deaths,date,time
0,AC,42,0,2020-03-30,19:42
1,AL,17,0,2020-03-30,19:43
2,AP,8,0,2020-03-30,19:44
3,AM,151,1,2020-03-30,19:43
4,BA,176,1,2020-03-30,19:44
5,CE,372,5,2020-03-30,19:46
6,DF,312,1,2020-03-30,19:45
7,ES,72,0,2020-03-30,19:45
8,GO,61,1,2020-03-30,19:46
9,MA,23,1,2020-03-30,19:45


In [172]:
# Carrega a série histórica e junta com o dia atual, removendo duplicadas
final = pd.read_csv("../dados/corona_brasil.csv").append(df,ignore_index=True)

final

,date,cases,deaths,uf,time
0,2020-01-30,0,0,CE,16:00
1,2020-01-30,0,0,MG,16:00
2,2020-01-30,0,0,RJ,16:00
3,2020-01-30,0,0,SP,16:00
4,2020-01-30,0,0,PR,16:00
...,...,...,...,...,...
988,2020-03-30,16,0,RR,19:51
989,2020-03-30,197,1,SC,19:52
990,2020-03-30,16,0,SE,19:50
991,2020-03-30,1517,113,SP,19:51


In [173]:
# Exporta o CSV

final.to_csv("../dados/corona_brasil.csv", index = False)